In [1]:
import findspark 
findspark.init("/Users/valentinaporcu/spark/spark-2.4.1-bin-hadoop2.7")
import pyspark 
from pyspark.sql import DataFrameNaFunctions 
from pyspark.sql.functions import lit 
from pyspark.ml.feature import StringIndexer  
from pyspark.ml import Pipeline 
from pyspark.sql import SparkSession
from pyspark.sql import functions
import pandas as pd
import numpy as np

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("alberi di decisione") \
    .config("spark.executor.memory", "6gb") \
    .getOrCreate()

In [ ]:
df = spark.read.csv("/Users/valentinaporcu/Dropbox/topic/12.\ Guida\ ai\ Big\ Data\ con\ Python/codice\ -\ guida\ ai\ big\ data\ con\ Python/Sezione\ 3/winsconsin_breast_cancer", 
                    inferSchema=True, header=True)

In [ ]:
df.head()

In [ ]:
df.printSchema()

In [ ]:
df.columns

In [ ]:
df.show()

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols = ['Clump_Thickness',
 'Uniformity_of_Cell_Size',
 'Uniformity_of_Cell_Shape',
 'Marginal_Adhesion',
 'Single_Epithelial_Cell_Size',
 'Bland_Chromatin',
 'Normal_Nucleoli',
 'Mitoses'], outputCol = "features")

In [ ]:
output = assembler.transform(df)

In [ ]:
output.show()

In [ ]:
output.select("features").show()

In [ ]:
transformed_df = output.select('features','Class')

In [ ]:
train, test = transformed_df.randomSplit([0.7,0.3])

In [ ]:
train.show()

In [ ]:
train.describe('Class').show()

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
decision_tree = DecisionTreeClassifier(featuresCol = 'features', 
                                       labelCol = 'Class')

In [ ]:
decision_tree_model = decision_tree.fit(train)

In [ ]:
dt_predictions = decision_tree_model.transform(test)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', 
                                          labelCol='Class')

In [ ]:
dt_acc = evaluator.evaluate(dt_predictions)

In [ ]:
dt_acc

In [ ]:
dt_predictions.show()